In [ ]:
# import libraries first
import duckdb
import pandas as pd 
import numpy as np 
import seaborn as sns
import plotly.express as px
from dotenv import load_dotenv
import os
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
from scipy import stats
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
# get your MotherDuck token from env
load_dotenv('.env')

token = os.getenv('motherduck_token')
print(token)

In [ ]:
# connect to the database
con = duckdb.connect(f"md:?motherduck_token={token}")

# define your SQL query
sql_query = "SELECT * FROM stocks_clouddb.msft_data"

# execute the query and fetch the result into a DataFrame
df = con.sql("SELECT * FROM stocks_clouddb.msft_data").fetchdf().copy()

In [ ]:
# display the first few rows of the dataset
print("First few rows of the dataset:")
print(df.head())

In [ ]:
df_close=df['close_price']

Check for Stationarity of Time Series 

In [ ]:
def test_stationarity(timeseries):
    # determing rolling statistics 
    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()
    # plot rolling statistics 
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black',label ='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean and Standard Deviation')
    plt.show(block=False)

    print("Results of Dickey fuller test") 
    adft= adfuller(timeseries, autolag ='AIC')
    # output for dft won't define what the values are
    # hence we manually write what values does it explain using a for loop
    output = pd.Series(adft[0:4],index=['Test Statistics','p-value','No. of lags used','Number of observations used'])
    for key,values in adft[4].items():
        output['critical value (%s)'%key] =  values
    print(output)
test_stationarity(df_close)
   

In [ ]:
# to separate the trend and the seasonality from a time series
# decompose the series
result = seasonal_decompose(df_close, model='multiplicative', period = 30)
fig = plt.figure()  
fig = result.plot()  
fig.set_size_inches(16, 9)

In [ ]:
# if it's not stationary then eliminate trend
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
df_log = np.log(df_close)
moving_avg = df_log.rolling(12).mean()
std_dev = df_log.rolling(12).std()
plt.legend(loc='best')
plt.title('Moving Average')
plt.plot(std_dev, color ="black", label = "Standard Deviation")
plt.plot(moving_avg, color="red", label = "Mean")
plt.legend()
plt.show()

In [ ]:
# split data into train and training set and visualize it 
train_data, test_data = df_log[3:int(len(df_log)*0.9)], df_log[int(len(df_log)*0.9):]
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('Closing Prices')
plt.plot(df_log, 'green', label='Train data')
plt.plot(test_data, 'blue', label='Test data')
plt.legend()

Auto ARIMA Model 

In [ ]:
model_autoARIMA = auto_arima(train_data, start_p=0, start_q=0,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
print(model_autoARIMA.summary())
model_autoARIMA.plot_diagnostics(figsize=(15,8))
plt.show()

In [ ]:
# modeling
model = ARIMA(train_data, order=(1,1,2))  
fitted = model.fit()  
print(fitted.summary())

In [ ]:
# forecast
fc = fitted.forecast(steps=len(test_data), alpha=0.05)  # Ensure fc has the same length as test_data


In [ ]:
# make as pandas series
fc_series = pd.Series(fc, index=test_data.index)

In [ ]:
# calculate the lower and upper bounds of the confidence interval manually
forecast_std = fitted.get_prediction(start=0, end=len(train_data) - 1).se_mean
alpha = 0.05  # 95% confidence interval
z = stats.norm.ppf(1 - alpha / 2)
lower_series = fc - z * forecast_std
upper_series = fc + z * forecast_std


In [ ]:
lower_series = pd.Series(lower_series, index=test_data.index)
upper_series = pd.Series(upper_series, index=test_data.index)


In [ ]:
# plot
plt.figure(figsize=(10,5), dpi=100)
plt.plot(train_data, label='training data')
plt.plot(test_data, color = 'blue', label='Actual Stock Price')
plt.plot(fc_series, color = 'orange',label='Predicted Stock Price')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.10)
plt.title('MSFT Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('MSFT Stock Price')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
# report performance
mse = mean_squared_error(test_data, fc)
print('MSE: '+str(mse))
mae = mean_absolute_error(test_data, fc)
print('MAE: '+str(mae))
rmse = math.sqrt(mean_squared_error(test_data, fc))
print('RMSE: '+str(rmse))
mape = np.mean(np.abs(fc - test_data)/np.abs(test_data))
print('MAPE: '+str(mape))

LSTM Model 

In [ ]:
import tensorflow as tf
import sklearn 
import keras
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout, LSTM
from sklearn.model_selection import TimeSeriesSplit


In [ ]:
data = df.filter(['close_price'])

In [ ]:
# convert the dataframe to a numpy array 
dataset = data.values 

In [ ]:
# get the number of rows to train the model on 
training_data_len = math.ceil(len(dataset)*0.8)
training_data_len

In [ ]:
# scale the data 
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

In [ ]:
# create the scaled training dataset 
train_data = scaled_data[0:training_data_len, :]

# split the data into x_train and y_train dataset 
X_train=[]
y_train =[]

for i in range(60, len(train_data)):
    X_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])
    # set 60 as the time step 
    if i < 61: 
        print(X_train)
        print(y_train)
        print()

In [ ]:
# convert the x_train and y_train to numpy arrays 
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
# reshape the data
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
# build the LSTM Model
model = Sequential()
model.add(LSTM(50, return_sequences=True , input_shape = (X_train.shape[1],1))) 
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
# compile model 
model.compile(optimizer='adam', loss='mse')

In [ ]:
# model training 
history = model.fit(X_train, y_train, epochs =25, batch_size = 8, verbose =1, shuffle = False)

In [ ]:
loss_per_epoch = history.history['loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch)

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss per Epoch')
plt.show()

In [ ]:
# create the testing dataset 
# create a new array containing scaled values 
test_data = scaled_data[training_data_len -60:, :]

# create the dataset x_test and y_test 
x_test =[]
y_test = dataset[training_data_len:,:]

for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i,0])


In [ ]:
# convert the data to a numpy array
x_test = np.array(x_test )


In [ ]:
# reshape the data 
x_test= np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))


In [ ]:
# get the model predicted price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
# get the root mean squared error (RMSE)
rmse= np.sqrt(np.mean(predictions -y_test)**2) 
rmse 

In [ ]:
# plot the data 
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions 

# visualize the data 
plt.figure (figsize = (16,8))
plt.title('Model ')
plt.xlabel('Data', fontsize =18)
plt.ylabel('Close Price USD $', fontsize=18)

plt.plot(train['close_price'])
plt.plot(valid[['close_price', 'Predictions']])
plt.legend(['Train','Valid','Predictions'], loc='lower right')
plt.show()

In [ ]:
# show the valid and predicted prices 
valid 